In [2]:
#coding:utf-8
import pandas as pd
import numpy as np
import copy
import os
import datetime
import time
raw_train_data_path = "/Users/mikeleung/Downloads/Metro_train/"
raw_test_data_path = "/Users/mikeleung/Downloads/Metro_testA/testA_record_2019-01-28.csv"
raw_submit_data_path = "/Users/mikeleung/Downloads/Metro_testA/testA_submit_2019-01-29.csv"
result_path = "/Users/mikeleung/Downloads/Metro_testA/testA_submit_2019-01-29.csv"

raw_train_files = os.listdir(raw_train_data_path)
raw_submit_df = pd.read_csv(raw_submit_data_path, engine='python')

readtime=time.clock()
raw_test_df = pd.read_csv(raw_test_data_path, engine='python').loc[:,['time','stationID','status']]
readtime2=time.clock()
print 'read time is ',readtime2-readtime


read time is  19.179059


In [3]:
raw_test_df['stationID'] = raw_test_df['stationID'].astype(np.uint16)
raw_test_df['status'] = raw_test_df['status'].astype(np.uint8)
raw_test_df['pep']=1 #每一条都代表一个人，用于sum()统计人数

In [4]:
slices=[] #用于pandas df切片，单独处理每个切片比处理一个大切片速度更快
for i in range(raw_test_df.shape[0]/10000+1):
    if i ==raw_test_df.shape[0]/10000:
        slices.append(copy.deepcopy(raw_test_df[i*10000:]))
        break
    slices.append(copy.deepcopy(raw_test_df[i*10000:(i+1)*10000]))

In [20]:
t0 = time.clock()
for i in range(len(slices)):#把原始数据时间归并到submit时间范围
    for idx,row in slices[i].iterrows():
        t=datetime.datetime.strptime(row['time'],'%Y-%m-%d %H:%M:%S')
        slices[i].loc[idx,['time']]=str(t-datetime.timedelta(minutes=(t.minute)%10)-datetime.timedelta(seconds=t.second))
t1=time.clock()
print 'change time cost ',t1-t0

change time cost  1454.27942


In [22]:
slices[0].head()

,time,stationID,status,pep
0,2019-01-28 00:00:00,33,0,1
1,2019-01-28 00:00:00,63,0,1
2,2019-01-28 00:00:00,63,0,1
3,2019-01-28 00:00:00,39,0,1
4,2019-01-28 00:00:00,33,0,1


In [33]:
testdf=pd.concat(slices)#把slices整合到一块
t2=time.clock()
print 'concat cost ',t2-t1

concat cost  3.171937


In [34]:
#开始groupby统计
testdf = testdf.groupby([testdf['stationID'],testdf['time'] ,testdf['status']]).sum()
t3=time.clock()
print 'gourpby cost ',t3-t2
print 'testdf shape is ',testdf.shape

gourpby cost  0.376454
testdf shape is  (17632, 1)


In [35]:
testdf

pep
stationID time                status     
0         2019-01-28 00:00:00 0         1
          2019-01-28 00:10:00 0         2
          2019-01-28 04:30:00 0         1
          2019-01-28 05:30:00 1        22
          2019-01-28 05:40:00 1        14
          2019-01-28 05:50:00 0         1
                              1        53
          2019-01-28 06:00:00 1        35
          2019-01-28 06:10:00 1        43
          2019-01-28 06:20:00 0         1
                              1        76
          2019-01-28 06:30:00 0        25
                              1       142
          2019-01-28 06:40:00 0        23
                              1       132
          2019-01-28 06:50:00 0         9
                              1       184
          2019-01-28 07:00:00 0        14
                              1       197
          2019-01-28 07:10:00 0        11
                              1       221
          2019-01-28 07:20:00 0        28
                              1       330
          2019-01-28 07:30:00 0        40
                              1       361
          2019-01-28 07:40:00 0        38
                              1       361
          2019-01-28 07:50:00 0        56
                              1       443
          2019-01-28 08:00:00 0        47
...                                   ...
80        2019-01-28 21:10:00 1        18
          2019-01-28 21:20:00 0        57
                              1        28
          2019-01-28 21:30:00 0        38
                              1        23
          2019-01-28 21:40:00 0        56
                              1        15
          2019-01-28 21:50:00 0        57
                              1        10
          2019-01-28 22:00:00 0        51
                              1        15
          2019-01-28 22:10:00 0        34
                              1        10
          2019-01-28 22:20:00 0        45
                              1        17
          2019-01-28 22:30:00 0        18
                              1        15
          2019-01-28 22:40:00 0        28
                              1         3
          2019-01-28 22:50:00 0        31
                              1         3
          2019-01-28 23:00:00 0         7
                              1         1
          2019-01-28 23:10:00 0        12
          2019-01-28 23:20:00 0        13
          2019-01-28 23:30:00 0        15
                              1         4
          2019-01-28 23:40:00 0         4
                              1         2
          2019-01-28 23:50:00 0         1

[17632 rows x 1 columns]

In [37]:
slices2=[] #现在要把统计好的df同样做切片处理
for i in range(testdf.shape[0]/10000+1):
    if i ==testdf.shape[0]/10000:
        slices2.append(copy.deepcopy(testdf[i*10000:]))
        break
    slices2.append(copy.deepcopy(testdf[i*10000:(i+1)*10000]))
t4=time.clock()
print 'slices2 cost ',t4-t3

slices2 cost  0.0666109999997


In [38]:
#因为submit格式是固定的，所以按照统计处来的df逆推submit的idx,写入sum值即可
raw_submit_df['inNums']=0
raw_submit_df['outNums']=0
for i in range(len(slices2)):
    for group,sum in slices2[i].iterrows():
        stationID=int(group[0])
        t=datetime.datetime.strptime(group[1],'%Y-%m-%d %H:%M:%S')
        idx=stationID*144+t.hour*6+t.minute/10
        type_in_out=int(group[2])
        if type_in_out==1:
            raw_submit_df.loc[idx,['inNums']]=sum['pep']
        else:
            raw_submit_df.loc[idx,['outNums']]=sum['pep']
t5=time.clock()
print 'write submit cost' ,t5-t4

write submit cost 8.45202


In [40]:
raw_submit_df.to_csv('/Users/mikeleung/Downloads/Metro_testA/submit.csv',index=None,encoding='utf-8')